# Optimization of Newton Hessian System Solve

In [861]:
import cProfile
import itertools
import matplotlib.pyplot as plt
import numpy as np
import os
import pstats
import scipy.sparse
import scipy.linalg
import time
from numpy.linalg import norm
from pstats import SortKey

import eblp
import eblp.eblp as cov
import eblp.ccpd as ccpd

np.set_printoptions(linewidth=150, precision=3, suppress=False)

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [862]:
n = 4

k, l = eblp.linalg.lower_subscripts(n)
ind = -np.ones((n, n), dtype=int)
ind[k, l] = np.arange(len(k))

In [863]:
ind + 1

array([[ 1,  0,  0,  0],
       [ 2,  5,  0,  0],
       [ 3,  6,  8,  0],
       [ 4,  7,  9, 10]])

In [864]:
print(" \\\\\n".join([" & ".join(map('{:d}'.format, line)) for line in ind+1]))

1 & 0 & 0 & 0 \\
2 & 5 & 0 & 0 \\
3 & 6 & 8 & 0 \\
4 & 7 & 9 & 10


In [865]:
h = eblp.ccpd.LltFunctional(4)
j = h._dh.copy()
j.data = np.concatenate((100 * np.arange(1, 1 + n * (n + 1) // 2), h._cross_term_index + 1))[h._perm]
j.T.todense()

matrix([[ 100,    0,    0,    0,    0,    0,    0,    0,    0,    0],
        [   2,    1,    0,    0,    0,    0,    0,    0,    0,    0],
        [   3,    0,    1,    0,    0,    0,    0,    0,    0,    0],
        [   4,    0,    0,    1,    0,    0,    0,    0,    0,    0],
        [   0,  200,    0,    0,  500,    0,    0,    0,    0,    0],
        [   0,    3,    2,    0,    6,    5,    0,    0,    0,    0],
        [   0,    4,    0,    2,    7,    0,    5,    0,    0,    0],
        [   0,    0,  300,    0,    0,  600,    0,  800,    0,    0],
        [   0,    0,    4,    3,    0,    7,    6,    9,    8,    0],
        [   0,    0,    0,  400,    0,    0,  700,    0,  900, 1000]])

In [866]:
print(" \\\\\n".join([" & ".join(map(lambda x: '2 x_{{{}}}'.format(x // 100) if x >= 100 else ('x_{{{}}}'.format(x) if x > 0 else ''), line)) for line in np.array(j.T.todense())]))

2 x_{1} &  &  &  &  &  &  &  &  &  \\
x_{2} & x_{1} &  &  &  &  &  &  &  &  \\
x_{3} &  & x_{1} &  &  &  &  &  &  &  \\
x_{4} &  &  & x_{1} &  &  &  &  &  &  \\
 & 2 x_{2} &  &  & 2 x_{5} &  &  &  &  &  \\
 & x_{3} & x_{2} &  & x_{6} & x_{5} &  &  &  &  \\
 & x_{4} &  & x_{2} & x_{7} &  & x_{5} &  &  &  \\
 &  & 2 x_{3} &  &  & 2 x_{6} &  & 2 x_{8} &  &  \\
 &  & x_{4} & x_{3} &  & x_{7} & x_{6} & x_{9} & x_{8} &  \\
 &  &  & 2 x_{4} &  &  & 2 x_{7} &  & 2 x_{9} & 2 x_{10}


In [867]:
N = np.arange(2, 10)
L = np.array([eblp.ccpd.LltFunctional(n)._dh.nnz for n in N])
print(np.polyfit(N, L, 3))
N *(2*N + 1) *(N + 1) / 6 - L

[ 3.333e-01  5.000e-01  1.667e-01 -3.856e-13]


array([0., 0., 0., 0., 0., 0., 0., 0.])

In [868]:
h = eblp.ccpd.LltFunctional(30)
(h._dh.todense() != 0).sum(axis=0).max(), (h._dh.todense() != 0).sum(axis=1).max()

(58, 30)

In [869]:

print(h._dh.nnz, (h._dh.T.dot(h._dh)).nnz)

9455 26565


In [870]:
N = np.arange(2, 10)
f = lambda a: (a.T @ a).nnz
L = np.array([f(eblp.ccpd.LltFunctional(n)._dh.T) for n in N])
print(np.polyfit(N, L, 4))
#N *(2*N + 1) *(N + 1) / 6 - L
p = np.polyfit(N, L, 4)
Lfit = np.polyval(p, N)
print(norm(L - Lfit))

[1.667e-01 3.333e-01 3.333e-01 1.667e-01 2.532e-12]
5.40190669790301e-13


In [871]:
L

array([   7,   26,   70,  155,  301,  532,  876, 1365])

In [872]:
L - Lfit

array([-2.363e-13,  7.461e-14,  1.279e-13,  8.527e-14,  0.000e+00,  0.000e+00,  0.000e+00, -4.547e-13])

In [873]:
h._hessian

<216225x465 sparse matrix of type '<class 'numpy.float64'>'
	with 9455 stored elements in Compressed Sparse Row format>

In [912]:
n = 4
N = n*(n+1)//2
h = eblp.ccpd.LltFunctional(n)

x = np.ones(N,)
H = [None] * N
for r in range(N):
    H[r] = np.array(h.hessian(x)[r * N:(r + 1)*N].todense())
    if r in (0, 5, N-2, N-1):
        print(r + 1)
        #print(H[r])
        print(" \\\\\n".join([" & ".join(map(lambda x: str(int(x)) if x != 0 else '', line)) for line in H[r]]))
nnz = np.array([(H[r] != 0).sum() for r in range(N)])
print(nnz, sum(nnz))

1
2 &  &  &  &  &  &  &  &  &  \\
 &  &  &  &  &  &  &  &  &  \\
 &  &  &  &  &  &  &  &  &  \\
 &  &  &  &  &  &  &  &  &  \\
 &  &  &  &  &  &  &  &  &  \\
 &  &  &  &  &  &  &  &  &  \\
 &  &  &  &  &  &  &  &  &  \\
 &  &  &  &  &  &  &  &  &  \\
 &  &  &  &  &  &  &  &  &  \\
 &  &  &  &  &  &  &  &  & 
6
 &  &  &  &  &  &  &  &  &  \\
 &  & 1 &  &  &  &  &  &  &  \\
 & 1 &  &  &  &  &  &  &  &  \\
 &  &  &  &  &  &  &  &  &  \\
 &  &  &  &  & 1 &  &  &  &  \\
 &  &  &  & 1 &  &  &  &  &  \\
 &  &  &  &  &  &  &  &  &  \\
 &  &  &  &  &  &  &  &  &  \\
 &  &  &  &  &  &  &  &  &  \\
 &  &  &  &  &  &  &  &  & 
9
 &  &  &  &  &  &  &  &  &  \\
 &  &  &  &  &  &  &  &  &  \\
 &  &  & 1 &  &  &  &  &  &  \\
 &  & 1 &  &  &  &  &  &  &  \\
 &  &  &  &  &  &  &  &  &  \\
 &  &  &  &  &  & 1 &  &  &  \\
 &  &  &  &  & 1 &  &  &  &  \\
 &  &  &  &  &  &  &  & 1 &  \\
 &  &  &  &  &  &  & 1 &  &  \\
 &  &  &  &  &  &  &  &  & 
10
 &  &  &  &  &  &  &  &  &  \\
 &  &  &  &  &  &  &  &  &  

## Hessian Terms / PCG

In [878]:
def create_random_spd(n, noise: float = 1e-2):
    # Generate an SPD matrix and its Cholesky factor. Explicitly calculate Frobenius error.
    np.random.seed(0)
    g = eblp.linalg.random_spsd(n, 2) + 0.001 * np.eye(n)
    g /= norm(g)
    b = g + noise * np.eye(n)
    l = scipy.linalg.cholesky(b, lower=True)
    l_vector = eblp.linalg.unravel_lower(l)
    return g, l_vector

In [885]:
n = 10
g, l_vector = create_random_spd(n, noise=1e-1)
a = scipy.sparse.csr_matrix(eblp.ccpd.frobenius_error_to_matrix(n))
f = eblp.ccpd.MomentCholeskyFunctionalExplicit(a, g)

result = scipy.optimize.minimize(f.fun, l_vector, method="Newton-CG", jac=f.grad,
                                 hess=f.hessian, options={"xtol": 1e-3, "disp": False, "maxiter": 20})

result2 = scipy.optimize.minimize(f.fun, l_vector, method="BFGS", jac=f.grad,
                                options={"gtol": 1e-3, "disp": False, "maxiter": 20})

In [881]:
result

     fun: 7.736260358307411e-07
     jac: array([ 3.439e-06,  1.235e-06, -6.155e-06,  8.990e-07, -4.350e-06, -4.092e-06,  2.283e-06, -9.903e-06,  2.237e-05, -3.052e-05, -2.821e-06,
       -7.409e-06,  4.898e-06, -1.180e-05, -3.681e-05,  1.549e-08,  1.871e-05,  1.776e-05,  1.149e-05, -2.196e-06, -2.298e-06, -6.567e-06,
       -5.790e-06,  1.617e-06, -3.407e-05,  6.511e-05, -9.337e-05, -6.848e-07,  1.986e-06, -8.592e-06, -3.323e-06, -1.859e-06,  1.894e-05,
       -1.347e-05, -1.798e-05, -8.606e-05,  5.167e-06,  2.696e-04, -3.096e-04,  6.263e-04,  2.899e-05, -2.148e-06,  7.568e-05, -1.150e-04,
        1.862e-04,  2.709e-07,  1.997e-05, -3.030e-05,  5.432e-05,  8.318e-05, -8.062e-05,  1.825e-04,  1.772e-04, -3.235e-04,  7.682e-05])
 message: 'Optimization terminated successfully.'
    nfev: 16
    nhev: 15
     nit: 15
    njev: 16
  status: 0
 success: True
       x: array([ 0.488, -0.18 , -0.112, -0.008, -0.242,  0.033, -0.04 ,  0.034,  0.083, -0.001,  0.341, -0.141, -0.082, -0.053, -0.2

In [886]:
result2

      fun: 5.003521426061576e-05
 hess_inv: array([[  0.589,   0.189,   0.066, ...,  -0.049,  -0.316,   0.578],
       [  0.189,   0.946,  -0.041, ...,   0.169,  -0.13 ,   0.562],
       [  0.066,  -0.041,   0.964, ...,   0.187,   0.51 ,  -0.69 ],
       ...,
       [ -0.049,   0.169,   0.187, ...,   4.892,   4.425,  -2.782],
       [ -0.316,  -0.13 ,   0.51 , ...,   4.425,  12.763, -16.423],
       [  0.578,   0.562,  -0.69 , ...,  -2.782, -16.423,  28.937]])
      jac: array([-3.711e-04,  2.654e-04,  6.707e-04, -2.436e-04, -3.690e-04,  1.163e-04,  6.085e-05,  6.813e-05,  3.106e-04, -3.805e-04,  2.897e-04,
        1.404e-03, -8.756e-04, -5.561e-04, -4.193e-04,  2.564e-04, -3.108e-05,  8.819e-04, -4.222e-04, -5.604e-04,  1.965e-04, -4.555e-05,
        1.837e-04,  4.837e-04,  6.868e-05,  2.791e-04,  8.003e-05,  5.707e-05,  6.461e-04,  2.669e-05,  2.348e-04, -1.290e-04, -2.698e-04,
        6.010e-04,  1.084e-04,  4.692e-04, -8.908e-05, -7.220e-04, -6.092e-04, -1.823e-03,  1.203e-04, -1.5